In [5]:
from matplotlib import pyplot as plt
import torch_model
import torch.utils.data as Data
import numpy as np
import argparse
import librosa
import random
import torch
import os
import common as com
from pytorch_common import *

In [7]:

"""
    This script defines the training procedure of SVS-UNet

    @Author: SunnerLi
"""
device = torch.device('cuda')

class SpectrogramDataset(Data.Dataset):
    def __init__(self, path):
        self.path = path
        self.files = sorted(os.listdir(path))
        self.files = [name for name in self.files if 'spec' in name]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        # Load the spectrogram
        mix = np.load(os.path.join(self.path, 'mixture', self.files[idx]))
        voc = np.load(os.path.join(self.path, 'vocal', self.files[idx]))

        # Random sample
        start = random.randint(0, mix.shape[-1] - 128 - 1)
        mix = mix[1:,start:start + 128, np.newaxis]
        voc = voc[1:,start:start + 128, np.newaxis]
        mix = np.asarray(mix, dtype=np.float32)
        voc = np.asarray(voc, dtype=np.float32)

        # To tensor
        mix = torch.from_numpy(mix).permute(2, 0, 1)
        voc = torch.from_numpy(voc).permute(2, 0, 1)
        return mix, voc

# # =========================================================================================
# # 1. Parse the direction and related parameters
# # =========================================================================================
# """
#                                     Parameter Explain
#     --------------------------------------------------------------------------------------------
#         --train_folder      The root of the training folder. You can generate via data.py
#         --load_path         The path of pre-trained model
#         --save_path         The model path you want to save 
#         --epoch             How many epoch you want to train
#     --------------------------------------------------------------------------------------------
# """
# parser = argparse.ArgumentParser()
# parser.add_argument('--train_folder', type = str, default = './data/vocals')
# parser.add_argument('--load_path'   , type = str, default = 'result.pth')
# parser.add_argument('--save_path'   , type = str, default = 'result.pth')
# parser.add_argument('--epoch'       , type = int, default = 2)
# args = parser.parse_args()

# =========================================================================================
# 2. Training
# =========================================================================================
# Create the data loader
train_folder = "D:/B4/u-net/dev_data/car1/train"
loader = Data.DataLoader(
    dataset = SpectrogramDataset(train_folder),
    batch_size=1, num_workers=0, shuffle=True
)

load_path = "D:/B4/u-net/model"
# Load the pre-trained model
model = torch_model.UNet()
epoch = 150
save_path = "D:/B4/u-net/model_torch"

model = torch_model.UNet().to(device)
summary(device, model)
# # Train!
# for ep in range(epoch):
#     bar = tqdm_table(loader)
#     for i, (mix, voc) in enumerate(bar):
#         mix, voc = mix.cuda(), voc.cuda()
#         model.backward(mix, voc)
#         if i == len(bar) - 1:
#             info_dict = model.getLoss(normalize = True)
#         else:
#             info_dict = model.getLoss(normalize = False)
#         info_dict.update({'Epoch': ep})
#         bar.set_table_info(info_dict)
#     model.save(save_path)
# print("Finish training!")

# # train models
# dirs = com.select_dirs(param=params, mode='development')

# for target_dir in dirs:
#     target = str(target_dir).split('/')[-1]
#     print(f'==== Start training [{target}] with {torch.cuda.device_count()} GPU(s). ====')

#     files = com.file_list_generator(target_dir)

#     model = torch_model.UNet().to(device)
#     summary(device, model)
#     task2 = Task2Lightning(device, model, params, files)
#     trainer = pl.Trainer(max_epochs=150, gpus=torch.cuda.device_count())
#     trainer.fit(task2)
    
#     model_file = f'{params.model_directory}/model_{target}.pth'
#     torch.save(task2.model.state_dict(), model_file)
#     print(f'saved {model_file}.\n')

ValueError: num_samples should be a positive integer value, but got num_samples=0